In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, LSTM, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')

# Tokenize text data using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 100  # Maximum sequence length
X_train_text = tokenizer(train_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
X_test_text = tokenizer(test_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

# Convert labels to numerical format
label_mapping = {'bully-Spam': 0, 'not_bully-Spam': 1, 'bully-Ham': 2, 'not_bully-Ham': 3}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

# Define BERT model
def build_bert_model():
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    bert_model = TFBertModel.from_pretrained("bert-base-uncased")
    bert_outputs = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]

    return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=bert_outputs)

# Build BERT model
bert_model = build_bert_model()

# Combine BERT and LSTM models
def build_combined_model(bert_model):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    bert_output = bert_model([input_ids, token_type_ids, attention_mask])
    
    # Add LSTM layer
    lstm_output = LSTM(128)(bert_output)
    
    output = Dense(4, activation='softmax')(lstm_output)

    return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=output)

# Build combined model
combined_model = build_combined_model(bert_model)

# Compile combined model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed
combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train combined model with a smaller batch size
history = combined_model.fit(
    [X_train_text['input_ids'], X_train_text['token_type_ids'], X_train_text['attention_mask']], 
    y_train, 
    epochs=5, 
    batch_size=16, 
    validation_split=0.1
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/5
79/79 [==============================] - 479s 6s/step - loss: 0.7333 - accuracy: 0.7090 - val_loss: 0.3611 - val_accuracy: 0.9291
Epoch 2/5
79/79 [==============================] - 617s 8s/step - loss: 0.3413 - accuracy: 0.8906 - val_loss: 0.2431 - val_accuracy: 0.9291
Epoch 3/5
79/79 [==============================] - 625s 8s/step - loss: 0.1955 - accuracy: 0.9437 - val_loss: 0.1633 - val_accuracy: 0.9362
Epoch 4/5
79/79 [==============================] - 631s 8s/step - loss: 0.1105 - accuracy: 0.9651 - val_loss: 0.1482 - val_accuracy: 0.9291
Epoch 5/5
79/79 [==============================] - 628s 8s/step - loss: 0.0620 - accuracy: 0.9841 - val_loss: 0.0561 - val_accuracy: 0.9787


In [2]:
# Make predictions on test data
predictions = combined_model.predict([X_test_text['input_ids'], X_test_text['token_type_ids'], X_test_text['attention_mask']])

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the model
from sklearn.metrics import classification_report

# Convert test labels to numpy array
y_test_np = np.array(y_test)

# Print classification report
print(classification_report(y_test_np, predicted_labels))


14/14 [==============================] - 72s 4s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.88      0.94       115
           2       0.95      0.87      0.91       103
           3       0.88      0.97      0.92       198

    accuracy                           0.92       426
   macro avg       0.96      0.93      0.94       426
weighted avg       0.93      0.92      0.92       426

